In [1]:
initial_path = 'peptide-QML'
# initial_path = '..'

In [2]:
import sys
sys.path.append(initial_path)

%load_ext autoreload
%autoreload 2
from my_code import helper_classes as c
from my_code import pytorch_model as m
from my_code import quantum_nodes as q

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
# name of the notebook
name_notebook = "sweep_12h_h_Xqm"

In [4]:
sweep = c.Sweep(
    name_notebook, 
    initial_path=initial_path,
    description="Try to find the best measurement and number of layers for the quantum block.",
    repetitions=list(range(1, 3)),
    n_layers_quantum_block=[0,1,2,5,12,20],
    qubits_measurement=[1, 'all'],
)

In [5]:
sweep.save()

In [6]:
for sweep_point in sweep:

    # print the point of the sweep
    sweep.print_sweep_status(sweep_point['idx'])

    if sweep_point['n_layers_quantum_block'] > 0:
        # -- QUANTUM BLOCK -- #
        n_qubits = 12
        quantum_layer = q.circuit(
            n_qubits = n_qubits, 
            device = "default.qubit.torch",
            device_options = {'shots': None},
            embedding = q.parts.AngleEmbedding, 
            # embedding_ansatz = q.parts.Ansatz_11,
            block_ansatz = q.parts.Ansatz_11,
            final_ansatz = q.parts.Ansatz_11,
            measurement = q.parts.Measurement('Z', sweep_point['qubits_measurement']),
            # embedding_n_layers = 4,
            # different_inputs_per_layer = True,
            block_n_layers = sweep_point['n_layers_quantum_block'],
            # wrapper_qlayer = pw.QLayerEmpty,
        )
    else:
        quantum_layer = None

    # -- MODEL -- #
    class Model(m.Model):
        def __init__(self):
            super(Model, self).__init__()
            n_qubits_measured = 12 if sweep_point['qubits_measurement'] == 'all' else sweep_point['qubits_measurement']
            n_qubits_measured = 12 if quantum_layer is None else n_qubits_measured
            self.fc1 = m.nn.Linear(228, 228//2)
            self.fc2 = m.nn.Linear(228//2, 12)
            self.fc3 = quantum_layer() if quantum_layer is not None else None
            self.fc4 = m.nn.Linear(n_qubits_measured, n_qubits_measured) 
            self.fc5 = m.nn.Linear(n_qubits_measured, 1)

        def forward(self, x):
            x = m.torch.sigmoid(self.fc1(x))
            x = m.torch.sigmoid(self.fc2(x))
            if self.fc3 is not None:
                x = self.fc3(x)
            x = m.torch.sigmoid(self.fc4(x))
            x = self.fc5(x)
            return x

    model = Model()

    # set quantum layer
    model.set_quantum_layer(quantum_layer)

    # set the name and path of the model
    model.set_name_and_path(name_notebook, initial_path)

    #set sweep point
    model.set_sweep_point(
        sweep_uuid = sweep.uuid,
        sweep_point = sweep_point
    )

    # -- TRAIN -- #
    model.optimize_params(
        #save info
        save_model_info = True,
        description = 'sweep test with 12 amino acids and clasical QML',    
        #data
        data=c.Data.load(initial_path=initial_path, file_name='PET_SCORES_12'),
        # train options
        optimizer = m.optim.Adam,
        optimizer_options = {'lr': 0.01},
        num_epochs = 2,                
        batch_size = 32,
        # early stopping options
        stop_training_options = {
            'lookback_epochs': 5, 
            'threshold_slope': 0.001,
            'threshold_std_dev': 0.2
        },
        # metadata
        metadata = {}
    )

    # -- VALIDATION and PLOTS -- #
    model.plot_losses(fig_size=(6,6))
    model.validate(
        pct=1,
        add_to_results=True,
        plot=True,
        print_items=False
    )

    # -- SAVE RESULTS -- #
    model.save_results()

    # -- ADD DATA GENERATED TO SWEEP -- #
    sweep.add_data(
        idx=sweep_point['idx'],
        model_uuid=model.uuid,
        best_loss_train=model.results.loss_epoch.best,
        best_loss_test=model.results.loss_validation_epoch.best,
        loss_validation=model.mean_loss_validation,
        time_train=model.results.time_epoch.last,
    )
    sweep.save(csv=False) #TODO: save csv=False automatically when sweep is already saved ion csv

    



 --- SWEEP POINT 1/24: (1, 0, 1) ---
 --- parameters sweeping: ['repetitions', 'n_layers_quantum_block', 'qubits_measurement'] 



RuntimeError: mat1 and mat2 shapes cannot be multiplied (397780x12 and 1x1)